In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch as tc
import cupy as cp
from sklearn.model_selection import KFold

from model.ffnn import FFNN

tc.autograd.set_grad_enabled(False)
        
if tc.cuda.is_available():
    print("CUDA tersedia!")
else:
    print("CUDA tidak tersedia!")
    

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
tc.set_default_device(device)
print("Using CUDA device:", tc.cuda.get_device_name(device))


# cp.cuda.Device(0).use()




CUDA tersedia!
Using CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype(np.float64)
y = mnist.target.astype(np.int32)


X = X / 255.0
# print(X)
print("Dataset MNIST_784 dimuat: X shape:", X.shape, " y shape:", y.shape)

Dataset MNIST_784 dimuat: X shape: (70000, 784)  y shape: (70000,)


In [3]:

subset_size = 70000
indices = np.random.choice(np.arange(X.shape[0]), subset_size, replace=False)
X_subset = X.iloc[indices]
y_subset = y.iloc[indices]

X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)


print("Data subset: Training:", X_train.shape, "Testing:", X_test.shape)

Data subset: Training: (56000, 784) Testing: (14000, 784)


In [4]:
def one_hot(labels, num_classes):
    oh = np.zeros((labels.shape[0], num_classes))
    oh[np.arange(labels.shape[0]), labels] = 1.0
    return oh

num_classes = 10
y_train_oh = one_hot(y_train, num_classes)
y_test_oh = one_hot(y_test, num_classes)

In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=100000, random_state=42, learning_rate_init=0.001, batch_size=200, verbose=True)

start_time = time.time()
mlp.fit(X_train, y_train)
sklearn_training_time = time.time() - start_time

#prediksi
y_pred_sklearn = mlp.predict(X_test)
accuracy_sklearn = np.mean(y_pred_sklearn == y_test)
print("Akurasi MLPClassifier:", accuracy_sklearn)
print("Waktu training MLPClassifier: {:.2f} detik".format(sklearn_training_time))

Iteration 1, loss = 0.41128634
Iteration 2, loss = 0.15991404
Iteration 3, loss = 0.11806751
Iteration 4, loss = 0.09080152
Iteration 5, loss = 0.07335404
Iteration 6, loss = 0.05839576
Iteration 7, loss = 0.05112221
Iteration 8, loss = 0.03947486
Iteration 9, loss = 0.03333927
Iteration 10, loss = 0.02739846
Iteration 11, loss = 0.02317062
Iteration 12, loss = 0.02094208
Iteration 13, loss = 0.01562742
Iteration 14, loss = 0.01598547
Iteration 15, loss = 0.01317096
Iteration 16, loss = 0.01286978
Iteration 17, loss = 0.01060070
Iteration 18, loss = 0.00757203
Iteration 19, loss = 0.01167275
Iteration 20, loss = 0.01327628
Iteration 21, loss = 0.01182487
Iteration 22, loss = 0.00590816
Iteration 23, loss = 0.00689852
Iteration 24, loss = 0.00248638
Iteration 25, loss = 0.00129375
Iteration 26, loss = 0.00072884
Iteration 27, loss = 0.01730916
Iteration 28, loss = 0.01463268
Iteration 29, loss = 0.00388358
Iteration 30, loss = 0.00634131
Iteration 31, loss = 0.00587968
Iteration 32, los

In [6]:
# X_train = cp.asarray(X_train)
# y_train_oh = cp.asarray(y_train_oh)
# X_test = cp.asarray(X_test)
# y_test = cp.asarray(y_test)

# custom_model = FFNN(
#     [784, 64, 10],
#     activations_list=['relu', 'sigmoid'],
#     loss_function='mse',  
#     weight_init='random_uniform',
#     init_params={'lower': -1, 'upper': 1}
# )

# epochs_custom = 10000  
# batch_size_custom = 200
# lr_custom = 0.001

# start_time = time.time()
# history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=False)
# custom_training_time = time.time() - start_time
# print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# # Prediksi pada data test
# y_pred_custom = custom_model.forward(X_test)
# # y_pred_custom.data adalah NumPy array, sehingga gunakan np.argmax dengan parameter axis
# y_pred_custom_labels = cp.argmax(y_pred_custom, axis=1)
# accuracy_custom = cp.mean(y_pred_custom_labels == y_test)
# print("Akurasi model custom FFNN:", accuracy_custom)

In [ ]:
X_train = np.asarray(X_train)
y_train_oh = np.asarray(y_train_oh)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

k_folds = 5 
epochs_custom = 10000  
batch_size_custom = 200
lr_custom = 0.001

kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)


fold_accuracies = []
fold_losses = []

print(f"Melakukan {k_folds}-Fold Cross Validation...\n")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"Fold {fold+1}/{k_folds}")

    X_train_fold = X_train[train_idx]
    y_train_fold = y_train_oh[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train_oh[val_idx]
    
    X_train_fold = tc.tensor(X_train_fold, dtype=tc.float32)
    X_val_fold = tc.tensor(X_val_fold, dtype=tc.float32)
    y_train_fold = tc.tensor(y_train_fold, dtype=tc.float32)
    y_val_fold = tc.tensor(y_val_fold, dtype=tc.int64)    

    custom_model = FFNN(
        [784, 256, 128, 64, 32, 10],
        activations_list=['relu', 'relu', 'relu', 'relu', 'softmax'],
        loss_function='cce',
        weight_init='he_xavier',
        init_params={'lower': -1, 'upper': 1}
    )

    history_custom = custom_model.train(
        X_train_fold, y_train_fold, 
        X_val_fold, y_val_fold,
        epochs=epochs_custom, 
        batch_size=batch_size_custom, 
        learning_rate=lr_custom, 
        verbose=False
    )

    y_pred_val = custom_model.forward(X_val_fold)
    y_pred_labels = tc.argmax(y_pred_val, dim=1)
    y_true_labels = tc.argmax(y_val_fold, dim=1)

    accuracy = (y_pred_labels == y_true_labels).float().mean().item()
    fold_accuracies.append(accuracy)

    val_loss = history_custom["val_loss"][-1] if "val_loss" in history_custom else None
    fold_losses.append(val_loss)

    print(f"✅ Fold {fold+1}: Akurasi = {accuracy:.8f}, Loss = {val_loss:.8f}")

# Rata-rata hasil Cross Validation
avg_accuracy = np.mean(fold_accuracies)
avg_loss = np.mean([l for l in fold_losses if l is not None])  

print(f"\n**Hasil K-Fold Cross Validation ({k_folds}-fold)**:")
print(f"-Rata-rata Akurasi: {avg_accuracy:.8f}")
print(f"-Rata-rata Loss: {avg_loss:.8f}")

Melakukan 5-Fold Cross Validation...

Fold 1/5
✅ Fold 1: Akurasi = 0.96973211, Loss = 0.12480148
Fold 2/5
✅ Fold 2: Akurasi = 0.97142857, Loss = 0.11974573
Fold 3/5
✅ Fold 3: Akurasi = 0.97258925, Loss = 0.10976908
Fold 4/5
✅ Fold 4: Akurasi = 0.97241068, Loss = 0.12350947
Fold 5/5
✅ Fold 5: Akurasi = 0.97098213, Loss = 0.11264258

📌 **Hasil K-Fold Cross Validation (5-fold)**:
➡️ Rata-rata Akurasi: 0.97142855
➡️ Rata-rata Loss: 0.11809367


In [8]:


# X_train = tc.tensor(np.asarray(X_train), dtype=tc.float32)
# X_test = tc.tensor(np.asarray(X_test), dtype=tc.float32)
# y_train_oh = tc.tensor(np.asarray(y_train_oh), dtype=tc.float32)
# y_test = tc.tensor(np.asarray(y_test), dtype=tc.int64)

# print(X_train.get_device())

# # Inisialisasi model custom FFNN
# custom_model = FFNN(
#     [784, 256, 128, 64, 32, 10],
#     activations_list=['relu', 'relu', 'relu', 'relu', 'softmax'],
#     loss_function='cce',  
#     weight_init='he_xavier',
#     init_params={'lower': -1, 'upper': 1}
# )
# epochs_custom = 100000
# batch_size_custom = 200
# lr_custom = 0.001

# start_time = time.time()
# history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=True)
# custom_training_time = time.time() - start_time
# print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# # Prediksi 
# y_pred_custom = custom_model.forward(X_test)
# y_pred_custom_labels = tc.argmax(y_pred_custom.data, dim=1)
# accuracy_custom = (y_pred_custom_labels == y_test).float().mean().item()
# print("Akurasi model custom FFNN:", accuracy_custom)


#107,32s tc cpu
#158,16s tc gpu
#215,52s cp gpu
#230,48s np cpu

#555.06 detik

#4000 20m

# Model custom selesai training dalam 1135.40 detik. GPU
# Akurasi model custom FFNN: 0.9748571515083313

# Model custom selesai training dalam 1193.06 detik. CPU
# Akurasi model custom FFNN: 0.9740714430809021

# Model custom selesai training dalam 733.88 detik.
# Akurasi model custom FFNN: 0.9735714197158813
